In [1]:
from helper_classes import PYKE
from helper_classes import Parser
from helper_classes import DataAnalyser
from helper_classes import PPMI
import itertools
import pickle
import pandas as pd
import util as ut
import numpy as np
import os
import re

In [2]:
random_state = 1
np.random.seed(random_state)

In [3]:

# DEFINE MODEL PARAMS
K = 45
num_of_dims = 50
bound_on_iter = 30
omega = 0.45557
e_release = 0.0414
final_embeddings = {}

In [4]:

kg_root = '/media/sami/70AE0846AE080776/folder2'
kg_path = kg_root + '/'

In [5]:

storage_path, experiment_folder = ut.create_experiment_folder()

parser = Parser(p_folder=storage_path, k=K)

parser.set_similarity_measure(PPMI)

model = PYKE()

analyser = DataAnalyser(p_folder=storage_path)
# For the illustration purpusoes lets only process first 5000 ntriples from each given file.
# To reproduce  reported results => parser.pipeline_of_preprocessing(kg_path)
holder = parser.pipeline_of_preprocessing(kg_path,bound=5000)



###### Preprocessing  starts ######


###### Constructing Inverted Index  starts ######
Number of RDF triples: 22367362
Number of vocabulary terms:  303239
Number of subjects:  152063
Constructing Inverted Index  took  1058.7494292259216  seconds



###### Calculation of PPMIs  starts ######
Calculation of PPMIs  took  97.17543578147888  seconds

Preprocessing  took  1157.0186760425568  seconds



In [6]:

vocab_size = len(holder)
key_list = list(holder.keys())
embeddings = ut.randomly_initialize_embedding_space(vocab_size, num_of_dims)

#for i in range(len(key_list)):
#   embeddings[key_list[i]] = embeddings_values[i]1

learned_embeddings = model.pipeline_of_learning_embeddings(e=embeddings,
                                                           max_iteration=bound_on_iter,
                                                           energy_release_at_epoch=e_release,
                                                           holder=holder, omega=omega)



###### Generating Embeddings:  starts ######
EPOCH:  0
EPOCH:  1
EPOCH:  2
EPOCH:  3
EPOCH:  4
EPOCH:  5
EPOCH:  6
EPOCH:  7
EPOCH:  8
EPOCH:  9
EPOCH:  10
EPOCH:  11
EPOCH:  12
EPOCH:  13
EPOCH:  14
EPOCH:  15
EPOCH:  16
EPOCH:  17
EPOCH:  18
EPOCH:  19
EPOCH:  20
EPOCH:  21
EPOCH:  22
EPOCH:  23
EPOCH:  24

 Epoch:  24
System energy: -0.03499999999999983
Generating Embeddings:  took  82035.81564879417  seconds



In [7]:
column_values = pd.Series(key_list)
learned_embeddings.insert(loc=0, column='new_column', value=column_values)
final_embeddings = learned_embeddings.set_index('new_column').T.to_dict('list')

In [8]:
(pd.DataFrame.from_dict(data=final_embeddings, orient='index')
   .to_csv(storage_path +'/PYKE_50_embd.csv', header=True))

# To use memory efficiently
del holder
del embeddings

In [9]:
type_info = ut.deserializer(path=storage_path, serialized_name='type_info')
len(type_info)# denoted as \mathcal{S} in the paper

# get the index of objects / get type information =>>> s #type o
all_types = sorted(set.union(*list(type_info.values())))
len(all_types)# denoted as C in the paper

4

In [10]:
vocabulary = ut.deserializer(path=storage_path, serialized_name='vocabulary')
for i in all_types:
    print(vocabulary[i])

http://hm2.com/class#Article
http://hm2.com/class#Colour
http://hm2.com/class#Size
http://hm2.com/class#HM


In [2]:
# this cell extracts the url from url list which contanins the embeddings for an item


path_root = '/media/sami/70AE0846AE080776/folder2'


all_urls = []
for root, dirs, files in os.walk(path_root):
    for file in files:
        links = []
        to_Read = os.path.join(root, file)
        f = open(to_Read,"r")
        urls = f.readline()
        while(urls):
            links.append(re.findall('^<http:\/\/hm2\.com\/article#\d{1,18}>', urls)) 
            urls = f.readline()
        # convert list of list to list
        merged = list(itertools.chain.from_iterable(links))
        # remove duplicates
        unique_urls = list(set(merged))
        # remove special charecter < ,>
        removetable = str.maketrans('', '', '<>')
        rmv_specialchar = [s.translate(removetable) for s in unique_urls]
        f.close()
        # store in final list
        all_urls.append(rmv_specialchar)
        # empty the lists
        del links[:]
        del merged
        del unique_urls
        del rmv_specialchar

#print(all_urls)
pickle.dump( all_urls, open(storage_path+"/all_urllist.p", "wb" ) )


In [3]:
# checks the maximum no of products from the transaction list
#define the function#
def find_max_list(list):
    list_len = [len(i) for i in list]
    print(max(list_len))

#print output#
find_max_list(all_urls)


266


In [7]:
import collections
# this cell provides the list of producs for all the transactions
counts = []
# Iterate over the list and to count internal elements in lists 
count = 0
for listElem in all_urls:
    counts.append(len(listElem))                    
#print('Number of elements count : ', counts)
print('Number of transaction:',len(counts))

counter=collections.Counter(counts)
print('Frequency of items:',counter)

Number of transaction: 718484
Frequency of items: Counter({2: 276092, 3: 163718, 4: 97769, 5: 59306, 6: 36823, 7: 23303, 8: 15520, 9: 10307, 1: 8709, 10: 7151, 11: 4908, 12: 3683, 13: 2616, 14: 2008, 15: 1421, 16: 1079, 17: 764, 18: 635, 19: 500, 20: 397, 21: 301, 22: 218, 23: 195, 24: 142, 25: 125, 26: 105, 27: 98, 28: 79, 29: 54, 32: 42, 31: 41, 30: 40, 33: 29, 34: 27, 35: 21, 36: 20, 39: 17, 37: 14, 42: 14, 38: 13, 41: 10, 44: 10, 40: 9, 57: 8, 46: 8, 52: 7, 51: 6, 50: 6, 68: 5, 47: 5, 56: 5, 45: 4, 48: 4, 53: 4, 69: 3, 60: 3, 59: 3, 82: 3, 58: 3, 43: 3, 86: 3, 65: 3, 88: 3, 49: 3, 78: 3, 72: 2, 70: 2, 111: 2, 66: 2, 74: 2, 62: 2, 80: 2, 96: 2, 79: 2, 98: 1, 54: 1, 165: 1, 87: 1, 115: 1, 214: 1, 221: 1, 118: 1, 116: 1, 149: 1, 123: 1, 180: 1, 182: 1, 206: 1, 130: 1, 202: 1, 107: 1, 97: 1, 114: 1, 55: 1, 208: 1, 146: 1, 67: 1, 93: 1, 83: 1, 75: 1, 266: 1, 85: 1, 140: 1, 169: 1, 109: 1, 71: 1, 64: 1, 105: 1, 91: 1, 77: 1, 102: 1, 61: 1})


In [8]:
# store transactions upto 10 products long


unusual_length=[]
intended_urls = [i for i in all_urls if len(i)< 11 and len(i)>1] # transaction length = 10, that is why <11

In [10]:
# this cell extracts the embeddings of the corrosponding items


final_list = []


for items in intended_urls:
    
    new_list = []
    for key in items:
        
        
        if key in final_embeddings:
           
            new_list.append(final_embeddings[key])
    final_list.append(new_list)
            
pickle.dump( final_list, open(storage_path+"/extracted_embedddings.p", "wb" ) )            

#print(final_list)



In [11]:
# this cell does the left padding for each transaction 

a = [None] * len(final_list)

# hardcoded the the maximum no of product in the trabsaction list as 5 need to change 
for i in range(0,len(final_list)):
    if len(final_list[i])<11:
        
        
        pad = 10 - len(final_list[i])
        a[i] = [[0]*50]*pad+(final_list[i])
    
#print(a)
pickle.dump( a, open(storage_path+"/padded_embedding.p", "wb" ) )


In [12]:
# Number of unique items in the entire data set, I have excluded transactions greter than 10 products but in below 
# cell the unique items from the entire data set is counted

r = re.compile("http:\/\/hm2\.com\/article#\d{1,18}")
newlist = list(filter(r.match, key_list)) # Read Note
print('The unique items from the entire data set:',len(newlist))

NameError: name 'key_list' is not defined

In [13]:
# this cell prepares the input data and output labels for final LSTM, here the x's are the input and y's are 
#the labels
x, y = list(), list()
nonecount = list()
#print('First padding',a[0])
for k,i in enumerate(a):
    if isinstance(i,list):
        
        seq_x, seq_y = i[:len(i) - 1], i[-1]
        x.append(seq_x)
        y.append(seq_y)
    else:
        nonecount.append(k)
print(len(nonecount))   


0


In [49]:
# storing final embedding dictionary

pickle.dump( final_embeddings, open(storage_path+"/Pyke50_pickle.p", "wb" ) )